Import libraries

In [45]:
import os
import sys
import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

from collections import defaultdict

Read in csv files

In [52]:
hitting = pd.read_csv('Hitting_16.csv')
# hitting.set_index('playerid', inplace=True)

fielding = pd.read_csv('Fielding_16.csv')
# fielding.set_index('playerid', inplace=True)
fielding.head()

,Name,Team,Pos,G,GS,Inn,PO,A,E,FE,...,DPT,DPF,Scp,SB,CS,PB,WP,FP,TZ,playerid
0,Neftali Feliz,PIT,P,62,0,53.2,1,5,0,0,...,0,0,NaN,3.0,NaN,NaN,NaN,1.000,NaN,18
1,Bartolo Colon,NYM,P,34,33,191.2,12,40,3,1,...,0,0,NaN,8.0,NaN,NaN,NaN,0.946,NaN,375
2,CC Sabathia,NYY,P,30,30,179.2,2,18,4,0,...,0,0,NaN,3.0,NaN,NaN,NaN,0.833,NaN,404
3,Juan Uribe,CLE,3B,68,68,551.2,43,128,7,3,...,2,1,NaN,NaN,NaN,NaN,NaN,0.961,NaN,454
4,Fernando Rodney,- - -,P,67,0,65.1,0,7,1,0,...,0,0,NaN,6.0,NaN,NaN,NaN,0.875,NaN,494


Filter the fielding dataset to get each player's primary position and add it as a column to the 'hitting' DataFrame

In [58]:
# remove pitchers
fielding = fielding[fielding.Pos != 'P']

# function that can be applied that will get positions
# based on number of games played there
def find_pos(df):
    positions = df.Pos
    d = defaultdict(int)
    for pos in positions:
        d[pos] += 1
    result = max(d.iteritems(), key=lambda x: x[1])
    return result[0]

positions = fielding.groupby('playerid').apply(find_pos)
positions = positions.reset_index()
positions = positions.rename(columns={0:'Pos'})
positions.merge(hitting, on='playerid', how='inner')

,playerid,Pos,Name,Team,G,PA,HR,R,RBI,SB,...,BABIP,AVG,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR
0,454,3B,Juan Uribe,Indians,73,259,7,19,25,0,...,0.227,0.206,0.259,0.332,0.256,53,-1.2,-15.9,8.3,0.1
1,589,RF,Carlos Beltran,- - -,151,593,29,73,93,1,...,0.315,0.295,0.337,0.513,0.358,124,-3.9,12.9,-10.8,2.3
2,629,LF,Brandon Barnes,Rockies,48,109,0,10,8,1,...,0.314,0.220,0.250,0.320,0.247,33,0.7,-8.2,4.5,0.0
3,639,3B,Adrian Beltre,Rangers,153,640,32,89,104,1,...,0.293,0.300,0.358,0.521,0.371,131,-0.8,22.7,15.2,6.1
4,746,C,A.J. Pierzynski,Braves,81,259,2,15,23,1,...,0.237,0.219,0.243,0.304,0.236,41,-3.3,-22.2,4.6,-1.0
5,785,3B,Todd Frazier,White Sox,158,666,40,89,98,15,...,0.236,0.225,0.302,0.464,0.326,105,0.8,4.5,-2.1,2.6
6,791,2B,Brandon Phillips,Reds,141,584,11,74,64,14,...,0.312,0.291,0.320,0.416,0.315,93,-3.6,-8.8,-0.4,1.0
7,950,LF,Marlon Byrd,Indians,34,129,5,11,19,0,...,0.347,0.270,0.326,0.452,0.328,102,0.9,1.3,1.2,0.7
8,971,SS,Jimmy Rollins,White Sox,41,166,2,25,8,5,...,0.270,0.221,0.295,0.329,0.277,71,0.3,-5.5,-0.1,0.0
9,1101,LF,Ichiro Suzuki,Marlins,143,365,1,48,22,10,...,0.329,0.291,0.354,0.376,0.320,100,1.3,1.2,0.9,1.4
